In [229]:
import numpy as np
from numpy.linalg import eig, inv, eigh, norm

import scipy as sc
from scipy.linalg import sqrtm
from scipy.spatial.distance import cdist

from sklearn import metrics, datasets
from sklearn.cluster import KMeans


## Trying to write a KMeans clustering that would work with complex vectors

In [256]:
class ComplexKMeans:
    def __init__(self, k_clusters=2, n_iter=500):
        self.k_clusters = k_clusters
        self.n_iter = n_iter
        self.is_fit = False
    
    distance_matrix = lambda self, x: np.array([[norm(vec - centroid) for centroid in self.centroids] for vec in x])
    
    def _assign_labels(self):
        is_changed = False
        for vec_num, vec_dists in enumerate(self.distances):
            new_label = vec_dists.argmin()
            if new_label != self.labels[vec_num]:
                is_changed = True
            self.labels[vec_num] = new_label
        return is_changed
    
    def fit(self, x):
        # initialize centroids and labels
        self.centroids = x[:self.k_clusters]
        self.labels = np.array([0] * len(x))
        
        for it in range(self.n_iter):
            # calculate distances and assign centroids
            self.distances = self.distance_matrix(x)
            is_changed = self._assign_labels()
            
            if not is_changed:
                self.is_fit = True
                break
            """
            print('---')
            print('Step {}: '.format(it))
            print('Labels: {}'.format(self.labels))
            print('Distances: {}'.format(self.distances))
            """
            
            # recalculate centroids
            for cl_num in range(self.k_clusters):
                vecs = [vec for ind, vec in enumerate(x) if self.labels[ind] == cl_num]
                self.centroids[cl_num] = np.average(vecs, axis=0)
        
        return self
    
    # not sure what I should do with these methods yet
    def predict(self):
        return self.labels
    
    def fit_predict(self, x):
        self.fit(x)
        return self.predict()

### Testing it on the iris dataset

Currently one of the points (`iris.data[50]`) gets a cluster label different from the `sklearn.cluster.KMeans` results

In [251]:
iris = datasets.load_iris()

In [258]:
iris_ckmeans = ComplexKMeans(k_clusters=3).fit(iris.data)
ckmeans_labels = iris_ckmeans.labels

iris_kmeans = KMeans(n_clusters=3).fit(iris.data)
kmeans_labels = iris_kmeans.labels_

In [259]:
print('ComplexKMeans score: {}'.format(metrics.adjusted_rand_score(iris.target, ckmeans_labels)))
print('KMeans score: {}'.format(metrics.adjusted_rand_score(iris.target, kmeans_labels)))

ComplexKMeans score: 0.6768052835197566
KMeans score: 0.6907610401119466


In [261]:
permutation = {0: 2, 1: 0, 2: 1}
np.array([permutation[i] for i in ckmeans_labels])

array([2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0])

In [254]:
kmeans_labels

array([2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0], dtype=int32)

In [263]:
[i for i, j in enumerate([k == permutation[l] for k, l in zip(kmeans_labels, ckmeans_labels)]) if j == False]

[50]

In [265]:
iris.data[50]

array([7. , 3.2, 4.7, 1.4])

## Spectral clustering of the graph itself

In [266]:
def spectral_clustering(adjacency):

    adj_matrix = np.matrix(adjacency)

    diag_degrees = np.diag([np.sum(i) for i in adj_matrix])
    sqrt_diag_degrees = sqrtm(inv(diag_degrees))

    laplacian = np.identity(len(adj_matrix)) - np.dot(np.dot(sqrt_diag_degrees, adj_matrix), sqrt_diag_degrees)
    #laplacian = diag_degrees - adj_matrix
    #laplacian = adj_matrix
    
    eigs = dict(zip(*eig(laplacian)))
    # find max eigenvalues
    eig_vals = list(eigs.keys())
    max_eig_val = max(eig_vals, key=abs)
    eig_vals.remove(max_eig_val)
    max2_eig_val = max(eig_vals, key=abs)
    
    to_cluster = np.array(list(map(lambda vec: np.array(vec)[0], [eigs[max_eig_val], eigs[max2_eig_val]]))).transpose()
    kmeans = KMeans(n_clusters=2, random_state=0).fit(to_cluster)
    return kmeans.labels_

In [268]:
l = 10
cluster_size = int(l / 2)
adjacency = np.array([[complex(1, 0)] * cluster_size + [complex(0, 0)] * cluster_size] * cluster_size +
                        [[complex(0, 0)] * cluster_size + [complex(1, 0)] * cluster_size] * cluster_size)


In [269]:
true_labels = [0] * cluster_size + [1] * cluster_size
pred_labels = spectral_clustering(adjacency)

metrics.adjusted_rand_score(true_labels, pred_labels)

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


0.09569377990430619